In [12]:
import time
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import StandardScaler

Data Loading

In [13]:
ratings_train = pd.read_csv("./Data/rate_train.csv", low_memory=False)
ratings_test = pd.read_csv("./Data/Ground_truth/groundtruth_0.3.csv",low_memory=False)

In [14]:
datasets_train = ratings_train.Node_Id.unique()
model_train = ratings_train.Model_Id.unique()
datasets_test = ratings_test.dataset.unique()
model_test = ratings_test.model.unique()
meta_models = pd.read_csv("./Data/model_v.csv",low_memory=False)
models = meta_models.model_id.unique()

Rating Matrix

In [15]:
data_model_train_matrix = pd.DataFrame(index=datasets_train,columns=models)
data_model_test_matrix = pd.DataFrame(index=datasets_test,columns=model_test)

In [16]:
for row in ratings_train.itertuples():
    data_model_train_matrix.loc[row[1]][row[2]] = row[3]


In [17]:
for row in ratings_test.itertuples():
    data_model_test_matrix.loc[row[1]][row[2]] = row[3]
data_model_test_matrix = data_model_test_matrix.fillna(0)

Dataset Similarity Matrix

In [18]:
meta_datasets = pd.read_csv("./Data/dataset_v.csv",low_memory=False)
datasets = meta_datasets.dataset_id.unique()
meta_datasets = meta_datasets.loc[:,("v1","v2","v3","v4","v5")]
scaler = StandardScaler()

# 对dataframe的数据进行标准化
scaled_data = scaler.fit_transform(meta_datasets)
# 将标准化后的数据转换为dataframe，并保留原始索引
scaled_df = pd.DataFrame(scaled_data, index=meta_datasets.index, columns=meta_datasets.columns)
meta_dataset_similarity = cosine_similarity(scaled_df.values.tolist())
meta_dataset_similarity = pd.DataFrame(meta_dataset_similarity,index=datasets,columns=datasets)

In [19]:
def cosine_similarity_func(ratings, user1, user2):
    # 找到两个用户共同评分的物品，并将这些评分放入一个向量中
    u1_ratings = ratings.loc[user1].dropna()
    u2_ratings = ratings.loc[user2].dropna()

    common_items = np.intersect1d(u1_ratings.index, u2_ratings.index).tolist()
    u1_common_ratings = u1_ratings.loc[common_items]
    u2_common_ratings = u2_ratings.loc[common_items]

    # 计算两个向量之间的余弦相似度
    if len(common_items) == 0:
        return 0
    else:
        cos_sim = np.dot(u1_common_ratings, u2_common_ratings) / (np.linalg.norm(u1_common_ratings) * np.linalg.norm(u2_common_ratings))
        return cos_sim

In [20]:
dataset_similarity = pd.DataFrame(index=datasets_train,columns=datasets_train)

In [21]:
start_time_train = time.time()

In [22]:
lambda_ = 0.5
for i in datasets_train:
    for j in datasets_train:
        rating_based_sim = cosine_similarity_func(data_model_train_matrix,i,j)
        if rating_based_sim != 0 and meta_dataset_similarity.loc[i][j] != 0:
            dataset_similarity.loc[i][j] = lambda_ * rating_based_sim + (1-lambda_) * meta_dataset_similarity.loc[i][j]
        else:
            dataset_similarity.loc[i][j] = 0


In [23]:
dataset_similarity

,0,29,14,28,16,17,13,12,10,38,...,275,274,279,278,280,281,288,286,287,285
0,1.0,0.098566,0.124857,0.126701,0.115517,0.13293,0.117093,0.122764,0.111825,0.109823,...,-0.021234,-0.009148,-0.013077,-0.004733,-0.042583,-0.055886,0.812771,0.811758,0.811325,0.813003
29,0.098566,1.0,0.965814,0.966359,0.964421,0.982425,0.983555,0.982679,0.989046,0.995141,...,0.244419,0.248923,0.249456,0.246419,0.295208,0.303703,0.21184,0.211555,0.211022,0.209916
14,0.124857,0.965814,1.0,0.997489,0.997701,0.962465,0.960072,0.960677,0.981173,0.965994,...,0.267301,0.270609,0.271789,0.26721,0.314695,0.326409,0.236114,0.235179,0.233865,0.234571
28,0.126701,0.966359,0.997489,1.0,0.998165,0.963593,0.95976,0.96041,0.980392,0.966075,...,0.267282,0.269679,0.26991,0.268026,0.317083,0.326415,0.237872,0.237363,0.236095,0.236248
16,0.115517,0.964421,0.997701,0.998165,1.0,0.962431,0.959373,0.960092,0.979771,0.964312,...,0.270424,0.271086,0.27273,0.268462,0.316068,0.328386,0.227722,0.227057,0.22589,0.225871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,-0.055886,0.303703,0.326409,0.326415,0.328386,0.291435,0.290603,0.289715,0.313903,0.300455,...,0.965017,0.966005,0.968387,0.962835,0.992795,1.0,0.145295,0.144876,0.145889,0.144497
288,0.812771,0.21184,0.236114,0.237872,0.227722,0.244641,0.229842,0.235642,0.224502,0.22247,...,0.08152,0.09323,0.089813,0.097427,0.156297,0.145295,1.0,0.999436,0.999152,0.999744
286,0.811758,0.211555,0.235179,0.237363,0.227057,0.243541,0.228608,0.234369,0.223552,0.221961,...,0.081978,0.093668,0.090225,0.097384,0.156139,0.144876,0.999436,1.0,0.999835,0.999073
287,0.811325,0.211022,0.233865,0.236095,0.22589,0.243167,0.2284,0.234479,0.222974,0.221513,...,0.083567,0.094867,0.091803,0.098052,0.156498,0.145889,0.999152,0.999835,1.0,0.998646


Prediction

In [24]:
def predict(rating_matrix, similarity_matrix):
    """
    根据评分矩阵和相似度矩阵预测评分。

    参数：
    rating_matrix (pd.DataFrame)：评分矩阵，包含NaN值
    similarity_matrix (numpy.array)：相似度矩阵

    返回：
    pd.DataFrame：预测评分矩阵
    """

    # 获取评分矩阵的均值（忽略NaN值）
    mean_rating = rating_matrix.mean(axis=1).values

    # 将评分矩阵中的NaN值替换为0
    rating_matrix_nan_to_zero = rating_matrix.fillna(0).values

    # 减去均值，得到归一化的评分矩阵
    normalized_rating_matrix = rating_matrix_nan_to_zero - mean_rating[:, np.newaxis]

    # 计算预测评分
    predicted_ratings = mean_rating[:, np.newaxis] + np.dot(similarity_matrix, normalized_rating_matrix) / np.abs(similarity_matrix).sum(axis=1)[:, np.newaxis]

    # 将预测评分数组转换为DataFrame
    predicted_ratings_df = pd.DataFrame(predicted_ratings, index=rating_matrix.index, columns=rating_matrix.columns)

    return predicted_ratings_df

In [25]:
def predict_ratings(rating_matrix, user_similarity_matrix, k=5):
    """
    输入：
    rating_matrix - 评分矩阵，DataFrame格式，其中NaN表示未评分
    user_similarity_matrix - 用户相似度矩阵，DataFrame格式
    k - 最近邻的数量，默认为5

    输出：
    prediction_matrix - 预测矩阵，DataFrame格式
    """

    # 初始化预测矩阵
    prediction_matrix = rating_matrix.copy()

    # 对于评分矩阵中的每个NaN值，使用K最近邻的方法预测评分
    for i in rating_matrix.index:
        for j in rating_matrix.columns:
            if np.isnan(rating_matrix.loc[i][j]):
                # 获取第i个用户的相似度值，并在相似度矩阵中找到K个最相似的用户
                similarity_values = user_similarity_matrix.loc[i].sort_values(ascending=False)[1:k+1]

                # 计算加权平均评分
                weighted_sum = 0
                similarity_sum = 0
                for index, value in similarity_values.items():
                    user_rating = rating_matrix.loc[index][j]
                    if not np.isnan(user_rating):
                        weighted_sum += value * user_rating
                        similarity_sum += value

                # 如果存在至少一个相似用户对该物品进行了评分，则计算预测评分
                if similarity_sum != 0:
                    prediction_matrix.loc[i][j] = weighted_sum / similarity_sum
                else:
                    # 如果没有相似用户评分，则使用当前用户的平均评分作为预测值
                    prediction_matrix.loc[i][j] = rating_matrix.loc[i].mean()

    return prediction_matrix

In [26]:
model_prediction_train = predict_ratings(data_model_train_matrix,dataset_similarity)

In [27]:
model_prediction_train = pd.DataFrame(model_prediction_train,index=datasets_train,columns=models).sort_index().sort_index(axis=1)

In [28]:
model_prediction_test = pd.DataFrame(index=datasets_test,columns=model_test)

In [29]:
end_time_train = time.time()

In [30]:
Training_time = end_time_train - start_time_train
Training_time

56.85710620880127

Metadata Based

In [43]:
def find_sim_index(index):
    row1 = meta_dataset_similarity.loc[index]
    row1_max_index = row1[row1 == row1.max()].index[0]
    return row1_max_index

In [66]:
def Find_Top_k(i,sim_matrix):
    row = sim_matrix.loc[i]
    row = row.sort_values(ascending=False)
    index_row = row.index
    index_row = index_row.values.tolist()
    return index_row

In [67]:
start_time = time.time()

In [68]:
for dataset in datasets_test:
    for model in model_test:
        dataset_sim_list = Find_Top_k(dataset,meta_dataset_similarity)[1:]
        # 仅保留存在于 model_prediction_train 的索引
        valid_indices = [idx for idx in dataset_sim_list if idx in model_prediction_train.index][:15]
        model_prediction_test.loc[dataset][model] = model_prediction_train.loc[valid_indices][model].mean()

In [69]:
valid_indices

[271, 270, 269, 281, 280, 266, 265, 264, 263, 262, 255, 256, 268, 276, 279]

In [70]:
end_time = time.time()

In [71]:
end_time - start_time

9.989575147628784

In [72]:
for i in datasets_test:
    for j in model_test:
        if data_model_test_matrix.loc[i][j] == 0:
            model_prediction_test.loc[i][j] = None

In [73]:
result = pd.DataFrame(columns={"dataset","model","f1_score","groundtruth_f1_score"})

In [74]:
for i in datasets_test:
    for j in model_test:
        if model_prediction_test.loc[i][j] is not None:
            f1_score = model_prediction_test.loc[i][j]
            groundtruth_f1_score = data_model_test_matrix.loc[i][j]
            result = result.append([{'dataset':i,'model':j,'f1_score':f1_score,'groundtruth_f1_score':groundtruth_f1_score}],ignore_index=True)

In [75]:
result

,model,dataset,groundtruth_f1_score,f1_score
0,463,93,0.300000,0.356665
1,465,93,0.300000,0.350868
2,363,93,0.909091,0.544820
3,367,93,0.303030,0.623460
4,369,93,0.303030,0.613331
...,...,...,...,...
8207,307,283,0.908689,0.684266
8208,317,283,0.882246,0.676334
8209,315,283,0.885276,0.678318
8210,303,283,0.751981,0.595143


In [76]:
result.to_csv("../Peak_Finding/Output/Dataset_only/Full_Dataset_only@15.csv",index=False)